In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time
import numpy.linalg as linalg
import cv2
import random
from mpl_toolkits.mplot3d import Axes3D
orb = cv2.ORB_create()
N = 100
s = 8

In [13]:
#function for computing the fundamental matrix
def computeF(xl, xr):
    A = []
    for i in range(len(xl)):
        x = xl[i]
        x_prime = xr[i]
        #Put a row into the matrix, with formula fropm class.
        A.append([x[0]*x_prime[0], x[1]*x_prime[0], x_prime[0],
                  x[0]*x_prime[1], x[1]*x_prime[1], x_prime[1], x[0], x[1], 1])
    #Get actual fundamental matrix.
    A = np.array(A)
    _, _, V = linalg.svd(A)
    F = V[-1,:].reshape((3,3))
    U, s, V = linalg.svd(F)
    sigma = np.diag(s)
    sigma[2,2] = 0
    return np.matmul(U, np.matmul(sigma, V))
    
def ransacF(points, kp1, kp2):
    n = 0 
    currF = None
    maxInliers = 0
    # run ransac
    for n in range(N):
        sample = random.sample(points, s)
        leftPoints = [kp1[point.trainIdx].pt for point in sample]
        rightPoints = [kp2[point.queryIdx].pt for point in sample]
        # compute our F matrix
        FToTest = computeF(leftPoints, rightPoints)
        numIn = 0
        for match in points:
            p = kp1[match.trainIdx].pt
            q = kp2[match.queryIdx].pt
            if (abs(np.matmul([p[0], p[1], 1], np.matmul(FToTest, [q[0], q[1], 1])))) < .01:
                numIn += 1
        if currF is None:
            currF = FToTest
        if numIn > maxInliers:
            maxInliers = numIn
            currF = FToTest
    return currF

In [11]:
npz = np.load("calibration/matrices.npz")
K = npz['cameraMatrix']
print(K.dtype)
K_prime = np.transpose(K)
W = np.matrix([[0, -1, 0], [1, 0, 0], [0, 0, 1]])
negW = np.transpose(W)
Z = np.matrix([[0, 1, 0], [-1, 0, 0], [0, 0, 0]])
trans = np.transpose
svd = linalg.svd
matmul = np.matmul
location = np.reshape((np.array([0, 0, 0, 1])), (4, 1))
# Compute null space of matrix A, algorithm from scipy mailing lists
def null(A, eps=1e-15):
    u, s, vh = linalg.svd(A)
    null_mask = (s <= eps)
    null_space = np.compress(null_mask, vh, axis=0)
    return trans(null_space)

def findPointsFront(points, R, t):
    # TODO find number of points in front of both cameras
    return 0

def updateLoc(F, points):
    global location
    # TODO
    # Need to find K for the camera
    print(F, type(F), F.dtype)
    E = matmul(K, matmul(F, K_prime))
    # https://en.wikipedia.org/wiki/Essential_matrix
    U, sigma, Vt = svd(E)
    t_mat = matmul(U, matmul(Z, trans(U)))
    RNeg = matmul(U, matmul(negW, Vt))
    RPos = matmul(U, matmul(W, Vt))
    opt = 0
    t = null(t_mat)
    neg_t = -1 * t
    # There are four valid comboes, we need to find the one with points in front of image
    # can probably parallelize this? should profile to see if worth
    numPoints1 = findPointsFront(points, RNeg, neg_t)
    numPoints2 = findPointsFront(points, RNeg, t)
    numPoints3 = findPointsFront(points, RPos, neg_t)
    numPoints4 = findPointsFront(points, RPos, t)
    whichNum = np.argmax([numPoints1, numPoints2, numPoints3, numPoints4])
    # currently grabs the first option by default
    if whichNum == 0:
        motion = np.hstack([RNeg, neg_t])
        motion = np.vstack([motion, np.array([0, 0, 0, 1])])
    if whichNum == 1: 
        motion = np.hstack(RNeg, t)
        motion = np.vstack([motion, np.array([0, 0, 0, 1])])
    if whichNum == 2:
        motion = np.hstack([RPos, neg_t])
        motion = np.vstack([motion, np.array([0, 0, 0, 1])])
    if whichNum == 3:
        motion = np.hstack([RPos, t])
        motion = np.vstack([motion, np.array([0, 0, 0, 1])])
    location = np.reshape((np.matmul(motion, location)), (4, 1))
    print(location)

float64


In [14]:
cap = cv2.VideoCapture('motion.mp4')
i = 0
last = None
lastDes = None
locations = []
while cap.isOpened():
    i += 1
    
    ret, frame = cap.read()
    if i % 10 != 0:
        continue
    if ret == False:
        break
    grayImg = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    kp, des = orb.detectAndCompute(grayImg, None)
    
    if last == None:
        last = kp
        lastDes = des
        continue
    
    # BFMatcher with default params
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(lastDes, des, k=2)
    # Apply ratio test
    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good.append(m)
    F = ransacF(good, kp, last)
    updateLoc(F, good)
    locations.append(location)
    last = kp
    lastDes = des


fig = plt.figure()
ax = fig.gca(projection='3d')
print(locations)
ax.plot([np.reshape(location, (4,))[0,0] for np.location in locations],
       [np.reshape(location, (4,))[0,1] for location in locations],
       [np.reshape(location, (4,))[0,2] for location in locations], label="locations")
plt.show()
    


[[  3.46896494e-07  -1.24198092e-06  -8.04441398e-04]
 [  1.73384094e-06   6.25632365e-08  -2.98058765e-03]
 [ -3.66472053e-04   2.21724557e-03   9.99992709e-01]] <class 'numpy.ndarray'> float64
[[-0.03475645]
 [ 0.0249308 ]
 [ 2.99978791]
 [ 1.        ]]
[[  1.21440806e-07  -1.58757619e-06   5.19754568e-04]
 [  2.18320437e-06   1.15915435e-06  -3.52281571e-03]
 [ -1.16402793e-03   1.40927997e-03   9.99991989e-01]] <class 'numpy.ndarray'> float64
[[ 0.02388802]
 [ 0.02981323]
 [ 1.99986439]
 [ 1.        ]]
[[  1.00665306e-06  -1.69505280e-06  -3.18956097e-04]
 [  1.16737882e-06  -6.69852238e-07  -9.63762042e-04]
 [ -1.94030658e-03   2.42665754e-03   9.99994658e-01]] <class 'numpy.ndarray'> float64
[[ 0.02752177]
 [-0.02810133]
 [ 2.99983504]
 [ 1.        ]]
[[  4.15546554e-07  -3.50685394e-07  -2.60426879e-04]
 [  9.32176469e-07  -1.94835060e-07  -1.06023085e-03]
 [ -1.11687343e-03   5.69502449e-04   9.99998618e-01]] <class 'numpy.ndarray'> float64
[[-0.02852803]
 [-0.03155025]
 [ 1.99

ValueError: Sample larger than population or is negative